In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
import tqdm.notebook as tq
import solitaire_env
import sys
import seaborn as sns
import pickle as pkl
import gp
sys.setrecursionlimit(10**6)

In [2]:
with open("games/run_2020-05-04 00:01:46.835706.pkl","rb") as file:
    run_data = pkl.load(file)

In [3]:
actions_matrix = run_data['actions_matrix']

In [4]:
iterations_used_per_game = run_data['iterations_used_per_game']
it = pd.DataFrame(data = {"iterations":iterations_used_per_game})['iterations'].value_counts()

In [5]:
max_action_take = max(it.index)
max_action_take

99

In [25]:
class replay_game:
    
    def __init__(self,gno=0):
        
        self.temp_en = solitaire_env.env()
        self.action_number = 0
        action_type = {0:'tableau_to_foundation_reveal',\
               1:'to_foundation_stack',2:'tableau_to_tableau_reveal',\
               3:'pile_to_tableau',\
                4:'foundation_to_tableau',\
               5:'tableau_to_tableau_not_reveal'\
              }
    
        for game_no,action_seq,end_state,start_state,state_seq,how_ends in actions_matrix:
            if game_no == gno:
                break;
        print("Reason game ended...   ",how_ends)
        print("Length of action sequence is.. ",len(action_seq))
        action_replay = "".join([str(x) for x in action_seq])
        action_replay = [action_type[int(x)] for x in action_replay]
        
        self.temp_en.state = start_state
        self.start_state = start_state
        
        self.end_state = end_state
        
        
        self.temp_en.state = self.temp_en.copy_state()
        
        self.state_seq = state_seq
        
        self.action_seq = action_replay
        
        print("starting state.. \n")
        
        print("***********************************pile*****************************************")
        self.temp_en.get_pile()
        print("\n\n\n\n\n\n\n\n")
        print("*****************************foundation*****************************************")
        
        self.temp_en.get_foundation()
        print("\n\n\n\n\n\n\n\n")
        print("******************************tableau*******************************************")
        
        self.temp_en.get_tableau()
        print("\n\n\n\n\n\n\n\n")
        
    def next_state(self):
        
        print("action number..  ", self.action_number)
        
        print("action taken..   ",self.action_seq[self.action_number])
        self.action_number +=1
        self.temp_en = solitaire_env.env()
        
        self.temp_en.state = self.state_seq[self.action_number]
        
        self.temp_en.state = self.temp_en.copy_state()
        
        print("starting state.. \n")
        
        print("***********************************pile*****************************************")
        self.temp_en.get_pile()
        print("\n\n\n\n\n\n\n\n")
        
        
        print("*****************************foundation*****************************************")
        
        self.temp_en.get_foundation()
        print("\n\n\n\n\n\n\n\n")
            
            
        print("******************************tableau*******************************************")
        
        self.temp_en.get_tableau()
        print("\n\n\n\n\n\n\n\n")
        
        
        
        
    def goto_state(self,n):
        self.action_number = n

In [26]:
game_replay = replay_game(gno = 636)

Reason game ended...    No action could be taken
Length of action sequence is..  99
starting state.. 

***********************************pile*****************************************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = spade      color = black      number =    10 speciality = None       face = up   
suit = heart      color = red        number =     1 speciality = ACE        face = up   
suit = heart      color = red        number =     4 speciality = None       face = up   
suit = club       color = black      number =     6 speciality = None       face = up   
suit = heart      color = red        number =     9 speciality = None       face = up   
suit = diamond    color = red        number =     6 speciality = None       face = up   
suit = spade      color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     8 speciality = None       face = up   
suit = spade   

In [91]:
game_replay.goto_state(50)
game_replay.next_state()

action number..   50
action taken..    to_foundation_stack
starting state.. 

***********************************pile*****************************************
suit = spade      color = black      number =     4 speciality = None       face = up   
suit = spade      color = black      number =     5 speciality = None       face = up   









*****************************foundation*****************************************
printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
suit = club       color = black      number =     5 speciality = None       face = up   
suit = club       color = black      number =     6 speciality = None       fa

In [89]:
game_replay.next_state()

action number..   57
action taken..    foundation_to_tableau
starting state.. 

***********************************pile*****************************************
suit = spade      color = black      number =     4 speciality = None       face = up   









*****************************foundation*****************************************
printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
suit = club       color = black      number =     2 speciality = None       face = up   
suit = club       color = black      number =     3 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
suit = club       color = black      number =     5 speciality = None       face = up   
suit = club       color = black      number =     6 speciality = None       face = up   
suit = club       color = black      number =     7 speciality = None       

In [ ]:



#tableau to tableau reveal not happening for actions where empty stack is found
#to_foundation was not priortised